Import libraries

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re


/Users/baohu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


request and parse to html file

In [51]:

URL_10k_p2_2023 = 'https://results.changeofpace.com/results.aspx?CId=16356&RId=6024'
response = requests.get(URL_10k_p2_2023)
response.raise_for_status()

### Develop get_ages(response)

In [52]:
soup = BeautifulSoup(response.content, "html.parser")
ages = soup.find_all('td' ,class_ = 'd-none d-sm-table-cell')
extracted_values = [age.get_text() for age in ages]
value_series = pd.Series(extracted_values)
value_series.head(20)

0            107
1              1
2             47
3              1
4     6:07 min/m
5            141
6              1
7             26
8              2
9     6:18 min/m
10           190
11             1
12            23
13             3
14    6:23 min/m
15           184
16             1
17            43
18             4
19    6:27 min/m
dtype: object

The accurate age values are: 71, 39, 26, 58... Upon close examination, it appears that the correct age sequence begins at index 3 and is interspersed with every 4 non-age values.

In [54]:
numeric_age_data = pd.to_numeric(value_series, errors='coerce').dropna()
correct_data = numeric_age_data.iloc[2::4]
correct_data.reset_index(drop=True, inplace=True)
correct_data = correct_data.astype(int)
correct_data.head()

0    47
1    26
2    23
3    43
4    41
dtype: int64

### Develop get_times(response)

In [20]:
soup = BeautifulSoup(response.content, "html.parser")
values = soup.find_all('td' ,class_ = 'd-none d-sm-table-cell ltw-time')
extracted_values = [time.get_text() for time in values]
value_series = pd.Series(extracted_values)
value_series.head(10)

0    53:00.21
1    53:00.21
2    53:07.43
3    53:02.35
4    53:17.77
5    53:07.10
6    53:17.56
7    53:09.97
8    53:38.30
9    53:19.90
dtype: object

similar to get_ages(), we only want the time value starting at index 1 and every other 1 value. In the actual dataset, the first time value is "gun time", and the second data is "net time", we only interested in "net time"

In [21]:
correct_data = value_series.iloc[1::2]
correct_data.reset_index(drop=True, inplace=True)
correct_data.head()

0    53:00.21
1    53:02.35
2    53:07.10
3    53:09.97
4    53:19.90
dtype: object

### Develop get_gender(response)

In [22]:
soup = BeautifulSoup(response.content, "html.parser")
values = soup.find_all('a', class_ = 'ltw-name')
extracted_values = [value.get_text() for value in values]
value_series = pd.Series(extracted_values)
value_series.head(6)

0                  Mike Cordano
1                     70-79 Yrs
2                          Male
3    Hortensia Cisneros Benftez
4                     30-39 Yrs
5                        Female
dtype: object

same, need to extract gender

In [23]:
correct_data = value_series.iloc[2::3]
correct_data.reset_index(drop=True, inplace=True)
correct_data.head()

0      Male
1    Female
2      Male
3      Male
4      Male
dtype: object

### Develop get_overall_pos(response)

In [37]:
soup = BeautifulSoup(response.content, "html.parser")
numeric_td_tags = soup.find_all('td', text=re.compile(r'\b\d+\b'))
extracted_values = [value.get_text() for value in numeric_td_tags]
value_series = pd.Series(extracted_values)
value_series.head(30)
correct_data = value_series.iloc[5::8]
correct_data.reset_index(drop=True, inplace=True)
correct_data.head()

/var/folders/32/3mmqs6sj1kb06qpdnfgb3cch0000gn/T/ipykernel_8122/2328599500.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  numeric_td_tags = soup.find_all('td', text=re.compile(r'\b\d+\b'))


0    51
1    52
2    53
3    54
4    55
dtype: object

### Develop get_last_name(response)

In [50]:
soup = BeautifulSoup(response.content, "html.parser")
values = soup.find_all('td', class_ = 'ltw-cell-left d-none d-sm-table-cell')
extracted_names = [name.get_text() for name in values]
value_series = pd.Series(extracted_names)
value_series.head(20)
correct_data = value_series.iloc[1::5]
correct_data.reset_index(drop=True, inplace=True)
correct_data.head()

0      Mcsorley
1       Michael
2    California
3    Sacramento
4          Male
dtype: object